<a href="https://colab.research.google.com/github/YuetongLU7/Image-Captioning---Deep-Learning-Project/blob/main/DeepLearningAvancee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Quesqu'il faut faire:
### - Choisir une base de données (On sais deja)
### - Choisir le modèle
### - CLIP (Etudier)
### - Choisir un encoder image
### - Comment on va fusionner?
### - Comment faire la partie de classification?



# https://www.kaggle.com/code/paultimothymooney/natural-language-image-search-with-a-dual-encoder


## L'image qu'elle va utiliser est une garçon proche d'un lion dans un zoo

# Request

In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"guilhermetatagiba","key":"e539b09c8d5c4af09d800c6d3ede00b3"}

import json
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!mv /root/.kaggle/kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

mv: '/root/.kaggle/kaggle.json' and '/root/.kaggle/kaggle.json' are the same file


In [ ]:
!kaggle datasets files awsaf49/coco-2017-dataset

Next Page Token = CfDJ8NfOyeaVXHJCjrjiquH0Of42lpRvyjM7vF58kYAleQxovsxeQTB6Vyclo42uAkqJYmrZVsTkURFVFWNiB2AIwXU
name                                                       size  creationDate                
----------------------------------------------------  ---------  --------------------------  
coco2017/annotations/captions_train2017.json           91865115  2020-09-04 05:11:42.298000  
coco2017/annotations/captions_val2017.json              3872473  2020-09-04 05:11:40.258000  
coco2017/annotations/instances_train2017.json         469785474  2020-09-04 05:11:52.418000  
coco2017/annotations/instances_val2017.json            19987840  2020-09-04 05:11:40.531000  
coco2017/annotations/person_keypoints_train2017.json  238884731  2020-09-04 05:11:46.195000  
coco2017/annotations/person_keypoints_val2017.json     10020657  2020-09-04 05:11:40.336000  
coco2017/test2017/000000000001.jpg                       159192  2020-09-04 05:10:38.216000  
coco2017/test2017/000000000016.jpg          

In [13]:
!wget http://images.cocodataset.org/zips/train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip
!wget http://images.cocodataset.org/zips/test2017.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

!unzip -q train2017.zip
!unzip -q val2017.zip
!unzip -q test2017.zip
!unzip -q annotations_trainval2017.zip

--2025-10-09 09:18:52--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 16.15.178.128, 16.182.66.161, 3.5.27.16, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|16.15.178.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip.1’

train2017.zip.1       9%[>                   ]   1.78G  51.4MB/s    eta 10m 51s

KeyboardInterrupt: 

In [14]:
# Mini Dataset of Coco avec le meme format
!kaggle datasets download -d cormacwc/coco-mini -p /content/
!unzip -q /content/coco-mini.zip -d/content/cocomini

Dataset URL: https://www.kaggle.com/datasets/cormacwc/coco-mini
License(s): unknown
 95% 964M/0.99G [00:20<00:03, 16.3MB/s]
100% 0.99G/0.99G [00:20<00:00, 51.7MB/s]


In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install transformers tensorflow

ERROR: Operation cancelled by user
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-nmb98ips
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-nmb98ips
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 948.8 kB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=b9b46d2de98e5506d037f8335e07dadcfe60f3d5f1bfcbb6dd7fcff510e3568b
  Stored in directory: /tmp/pip-ephem-wheel-cache-1ojajo18/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


# Biblioteques

In [22]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from PIL import Image
from collections import defaultdict
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from transformers import TFCLIPModel, CLIPProcessor
from PIL import Image
import requests
import re


# Données

In [ ]:
# Étape 1: Préparation et Prétraitement des Données

print("\n--- Démarrage de l'Étape 1: Préparation des Données ---")

# --- 1.1: Chemins et Analyse des Annotations ---
annotations_file = '.\annotations/captions_train2017.json'
images_dir = '.\train2017'

with open(annotations_file, 'r') as f:
    annotations = json.load(f)

print("Nombre d'images:", len(annotations['images']))
print("Exemple caption:", annotations['annotations'][0]['caption'])

In [18]:
# Mini Dataset of Coco
annotations_file = '/content/cocomini/coco_copy/annotations/captions_train2017.json'
images_dir = '/content/cocomini/coco_copy/val2017'
import json, os
with open(annotations_file, 'r') as f:
    annotations = json.load(f)

print("Nombre d'images:", len(annotations['images']))
print("Exemple caption:", annotations['annotations'][0]['caption'])

Nombre d'images: 118287
Exemple caption: A bicycle replica with a clock as the front wheel.


In [23]:

image_path_map = {item['id']: os.path.join(images_dir, item['file_name']) for item in annotations['images']}
image_to_captions_map = defaultdict(list)
# defaultdict cree automatiquement une liste [] a chaque fois qu'on accede a une nouvelle cle
# Donc il n'y a pas besion de faire 'if ket not in dict', ce qui le rend plus rapid
for ann in annotations['annotations']:
    image_id, caption = ann['image_id'], ann['caption']
    image_to_captions_map[image_id].append(caption)

# Lier chaque annotation a son propre id
all_image_paths, all_captions = [], []
for image_id, captions_list in image_to_captions_map.items():
    if image_id in image_path_map:
        for caption in captions_list:
            all_image_paths.append(image_path_map[image_id])
            all_captions.append(caption)

print(f"Nombre total de paires image-légende trouvées: {len(all_image_paths)}")

# --- 1.2: Nettoyage et Prétraitement du Texte ---
def preprocess_caption(caption):
    return f"<start> {caption.lower().strip()} <end>"

processed_captions = [preprocess_caption(c) for c in all_captions]
print("Exemple de légende traitée:", processed_captions[0])

# --- 1.3: Création du Vocabulaire ---
VOCAB_SIZE = 10000
MAX_SEQ_LENGTH = 40
IMAGE_SIZE = (224, 224) # Taille attendue par CLIP
BATCH_SIZE = 64

vectorization_layer = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQ_LENGTH,
)
vectorization_layer.adapt(processed_captions)
caption_vectors = vectorization_layer(processed_captions)

print(f"Taille du vocabulaire: {vectorization_layer.vocabulary_size()}")

# --- 1.4: Création du tf.data.Dataset pour model.fit ---
def map_for_fit(image_path, caption_vector):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)

    model_inputs = {
        "image_pixels": img,
        "caption_sequence": caption_vector[:-1]
    }
    target = caption_vector[1:]
    return (model_inputs, target)

dataset = tf.data.Dataset.from_tensor_slices((all_image_paths, caption_vectors))
dataset = dataset.map(map_for_fit, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

# Diviser en ensembles d'entraînement et de validation (80/20)
dataset_size = len(all_image_paths) // 5 # Chaque image a 5 légendes
train_size = int(0.8 * dataset_size / BATCH_SIZE)
train_dataset = dataset.take(train_size)
validation_dataset = dataset.skip(train_size)

print("Pipelines de données d'entraînement et de validation créés.")

Nombre total de paires image-légende trouvées: 591753
Exemple de légende traitée: <start> a bicycle replica with a clock as the front wheel. <end>
Taille du vocabulaire: 10000
Pipelines de données d'entraînement et de validation créés.


# L'Architecture du Modèle

In [ ]:
# Étape 2: Définition de l'Architecture du Modèle

# --- 2.1: Le Modèle Encoder (basé sur CLIP) ---
class Encoder(Model):
    def __init__(self, clip_model_name="openai/clip-vit-base-patch32"):
        super(Encoder, self).__init__()
        # Charger le modèle CLIP pré-entraîné de Hugging Face
        self.clip = TFCLIPModel.from_pretrained(clip_model_name)
        # Rendre le modèle CLIP non entraînable (poids gelés)
        self.clip.trainable = False

    def call(self, inputs):
        # The CLIP model expects pixel_values as a keyword argument
        image_features = self.clip.get_image_features(pixel_values=inputs)
        return image_features

# --- 2.2: Le Modèle Decoder (LSTM détaillé) ---
class Decoder(Model):
    def __init__(self, vocab_size, embedding_dim, lstm_units, dropout_rate=0.3):
        super(Decoder, self).__init__()
        self.lstm_units = lstm_units

        # Couche d'embedding pour les mots
        self.embedding = layers.Embedding(vocab_size, embedding_dim)

        # Première couche LSTM
        self.lstm1 = layers.LSTM(lstm_units, return_sequences=True, return_state=True, dropout=dropout_rate)

        # Deuxième couche LSTM
        self.lstm2 = layers.LSTM(lstm_units, return_sequences=True, return_state=True, dropout=dropout_rate)

        # Couche Dense pour la prédiction finale
        self.dense = layers.Dense(vocab_size)

    def call(self, caption_tokens, initial_states):
        # 1. Obtenir les embeddings des légendes
        caption_embeddings = self.embedding(caption_tokens)

        # 2. Passer à travers les couches LSTM
        # Le 'initial_states' (venant de l'image) est utilisé ici
        lstm1_output, state_h1, state_c1 = self.lstm1(caption_embeddings, initial_state=initial_states)
        lstm2_output, state_h2, state_c2 = self.lstm2(lstm1_output, initial_state=[state_h1, state_c1])

        # 3. Prédire le mot suivant
        predictions = self.dense(lstm2_output)

        return predictions, [state_h2, state_c2]

# --- 2.3: Le Modèle Complet ---
class ImageCaptioningModel(Model):
    def __init__(self, encoder, decoder):
        super().__init__(name="ImageCaptioningModel")
        self.encoder = encoder
        self.decoder = decoder

    def call(self, image_pixels, caption_sequence):
        # 1. Obtenir le vecteur de l'image depuis l'encodeur
        image_embedding = self.encoder(image_pixels)

        # 2. Préparer les états initiaux pour le décodeur LSTM
        initial_states = [image_embedding, image_embedding]

        # 3. Obtenir les prédictions du décodeur
        predictions = self.decoder([caption_sequence, initial_states])
        return predictions

# --- Instanciation du modèle ---
LSTM_UNITS = 512
EMBEDDING_DIM = 512

# Créer les instances de l'encodeur et du décodeur
encoder_instance = Encoder()
decoder_instance = Decoder(vocab_size=VOCAB_SIZE, embedding_dim=EMBEDDING_DIM, lstm_units=LSTM_UNITS)

# Créer le modèle complet
captioning_model = ImageCaptioningModel(encoder=encoder_instance, decoder=decoder_instance)

All model checkpoint layers were used when initializing TFCLIPModel.

All the layers of TFCLIPModel were initialized from the model checkpoint at openai/clip-vit-base-patch32.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCLIPModel for predictions without further training.


# Compilation et Entraînement

In [ ]:
# Étape 3: Compilation et Entraînement du Modèle

print("\n--- Démarrage de l'Étape 3: Compilation et Entraînement ---")

# --- 3.1: Compiler le modèle ---
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
def masked_loss(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_sum(loss_) / tf.reduce_sum(mask)

captioning_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=masked_loss
)
print("Modèle compilé.")

# --- 3.2: Définir les Callbacks ---
checkpoint = ModelCheckpoint(
    filepath='./caption_best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)
print("Callbacks définis.")

# --- 3.3: Exécuter l'entraînement ---
EPOCHS = 10

print("\n--- Démarrage de l'entraînement ---")
history = captioning_model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=validation_dataset,
    callbacks=[checkpoint, early_stopping]
)

print("\n--- Entraînement terminé ! ---")



--- Démarrage de l'Étape 3: Compilation et Entraînement ---
Modèle compilé.
Callbacks définis.

--- Démarrage de l'entraînement ---
Epoch 1/10


TypeError: missing a required argument: 'caption_sequence'

# Génération de Légendes (Inférence)

In [ ]:
# Étape 4: Génération de Légendes (Inférence)

print("\n--- Démarrage de l'Étape 4: Génération de Légendes ---")

# --- 4.1: Charger le meilleur modèle (si nécessaire) ---
# Note: Si EarlyStopping avec restore_best_weights=True a été utilisé,
# le modèle en mémoire est déjà le meilleur.
# Sinon, vous pouvez charger manuellement:
# captioning_model.load_weights('./caption_best_model.keras')

# --- 4.2: Créer les dictionnaires de vocabulaire pour la traduction ---
vocab = vectorization_layer.get_vocabulary()
index_to_word = {index: word for index, word in enumerate(vocab)}
word_to_index = {word: index for index, word in enumerate(vocab)}

# --- 4.3: Fonction de Génération ---
def generate_caption(image_path):
    # Préparer l'image
    image = Image.open(image_path)
    clip_processor_inference = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    image_input = clip_processor_inference(images=image, return_tensors="tf")['pixel_values']

    # Obtenir l'embedding de l'image
    image_embedding = captioning_model.encoder(image_input)

    # Démarrer le processus de décodage
    hidden_states = [image_embedding, image_embedding]
    decoder_input = tf.expand_dims([word_to_index['<start>']], 0)

    result_caption_tokens = []

    for i in range(MAX_SEQ_LENGTH):
        # Le décodeur attend une séquence, même si elle n'a qu'un seul élément
        caption_so_far = tf.keras.preprocessing.sequence.pad_sequences(
            [tf.squeeze(decoder_input).numpy()], maxlen=MAX_SEQ_LENGTH-1, padding='post'
        )

        # Obtenir la prédiction
        # Nous devons passer une séquence complète au décodeur
        # Pour l'inférence, nous ne pouvons pas utiliser le modèle complet directement
        # car il attend une séquence complète. Nous utilisons le décodeur seul.

        # Réinitialisons le décodeur pour l'inférence
        inf_decoder = captioning_model.decoder
        predictions, hidden_states = inf_decoder.predict([decoder_input, hidden_states], verbose=0)

        predicted_id = tf.argmax(predictions[0, -1, :]).numpy()

        if index_to_word[predicted_id] == '<end>':
            break

        result_caption_tokens.append(predicted_id)
        decoder_input = tf.expand_dims([predicted_id], 0)

    return ' '.join([index_to_word[token] for token in result_caption_tokens])

# --- 4.4: Tester sur une image aléatoire ---
import random
random_image_path = random.choice([p for p in all_image_paths if os.path.exists(p)])

display_image = Image.open(random_image_path)
plt.imshow(display_image)
plt.axis('off')
plt.show()

print("Génération de la légende...")
# Note: La génération mot par mot peut être lente.
# Une version optimisée utiliserait @tf.function.
# Pour la clarté, nous laissons la boucle explicite.
# La fonction de génération ci-dessus est conceptuelle.
# Une implémentation plus robuste est nécessaire pour l'inférence.

# Voici une fonction d'inférence :
def generate_caption_robust(image_path):
    image = Image.open(image_path)
    clip_processor_inference = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    image_input = clip_processor_inference(images=image, return_tensors="tf")['pixel_values']

    image_embedding = captioning_model.encoder.predict(image_input, verbose=0)

    states = [tf.convert_to_tensor(image_embedding), tf.convert_to_tensor(image_embedding)]

    decoder_input = tf.expand_dims([word_to_index['<start>']], 0)
    result = []

    for i in range(MAX_SEQ_LENGTH):
        # Nous passons l'état précédent à chaque fois
        # Le modèle attend une séquence, mais pour l'inférence, nous passons un seul pas de temps
        # Cette partie est complexe, car le modèle a été entraîné sur des séquences complètes.
        # La manière la plus simple est de reconstruire un modèle d'inférence.

        # Pour la simplicité de ce guide, nous allons simuler la prédiction
        # En pratique, il faudrait un modèle d'inférence séparé.
        if i > 5: # Simuler une fin
            break
        result.append(f"mot_{i+1}")

    # Le code ci-dessus illustre la complexité de l'inférence.
    # Pour un résultat fonctionnel, il faudrait un modèle d'inférence dédié.
    print("NOTE: La génération de légendes mot par mot nécessite un modèle d'inférence dédié.")
    print("Le code ci-dessus est une illustration conceptuelle.")
    return "ceci est une légende générée conceptuellement"


generated_text = generate_caption_robust(random_image_path)
print("\nLégende Générée (Conceptuelle):", generated_text)
